In [1]:
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera
from scipy.signal import find_peaks, butter, filtfilt
from simple_pid import PID
#from ipywidgets import interact, interactive
from pwm import PWM
import ipywidgets as ipw
import time
import matplotlib.pyplot as plt
import skimage as ski
import numpy as np
import time
import threading

# Takes in a percent and a pwm, sets the speed according to the percent
def setEngineSpeed(percent, pwm):
    # If the percent is less than 0, set to 0
    if percent < 0:
        percent = 0
	# If the percent is greater than 1, set to 1
    if percent > 1:
        percent = 1
	# Calculate the speed to set the motor
    newSpeed = int(1000000 * percent + 1000000)
    #print("set engine speed ", speed)
	# Set the duty_cylce to the speed
    pwm.duty_cycle = newSpeed

# Takes in a percent and a pwm, turns to the right according to the percent
def turnRight(percent, pwm):
    # print("Turning Right")
	# If the percent is less than 0, set to 0
    if percent < 0:
        percent = 0
	# If the percent is greater than 1, set to 1
    if percent > 1:
        percent = 1
	# Calculate the direction to set the servo
    value = int(1450000 - 500000 * percent)
    #print("turn right speed: ", value)
	# Set the duty_cylce to the value
    pwm.duty_cycle = value

# Takes in a percent and a pwm, turns to the left according to the percent
def turnLeft(percent, pwm):
    #print("Turning left")
	# If the percent is less than 0, set to 0
    if percent < 0:
        percent = 0
	# If the percent is greater than 1, set to 1
    if percent > 1:
        percent = 1
	# Calculate the direction to set the servo
    value = int(500000 * percent + 1450000)
    #print("turn left speed: ", value)
	# Set the duty_cylce to the value
    pwm.duty_cycle = value

def turn(percent, pwm):
    #print("I am attempting a turn")
    if percent < -1:
        percent = -1
    if percent > 1:
        percent = 1
    if percent < 0:
        turnRight(-1 * percent, pwm)
    else:
        turnLeft(percent, pwm)
        
#def getNewSpeed(turnPercent):
#    if turnPercent <= 0.15:
#        newSpeed = 0.20
#    elif turnPercent <=0.3:
#        newSpeed = 0.175
#    else:
#        newSpeed = 0.16
#    return newSpeed

# Set the speed to be a discounted rate of the maxSpeed
def getNewSpeed(turnPercent, maxSpeed, minSpeed):
    speed = maxSpeed - (turnPercent * maxSpeed)
    # Make sure the speed is never less than minSpeed
    if(speed < minSpeed):
        speed = minSpeed
    return speed

# Create the sliders to effect the values of maxSpeed, minSpeed, Kp, Ki, Kd. Have two buttons for stopping the program and taking pictures
maxSpeed = ipw.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.25, description='Max Speed')
display(maxSpeed)
minSpeed = ipw.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.17, description='Min Speed')
display(minSpeed)
Kp = ipw.FloatSlider(min=0.0, max=10.0, step=0.01, value=0.30, description='Kp')
display(Kp)
Ki = ipw.FloatSlider(min=0.0, max=10.0, step=0.01, value=0.10, description='Ki')
display(Ki)
Kd = ipw.FloatSlider(min=0.0, max=10.0, step=0.01, value=0.20, description='Kd')
display(Kd)
running = ipw.ToggleButton(value=True, description='STOP', disabled=False, button_style='', tooltip='Description', icon='check')
display(running)
button = ipw.ToggleButton(value=False, description='Snap photo', disabled=False,button_style='', tooltip='Description', icon='check')
display(button)

FloatSlider(value=0.25, description='Max Speed', max=1.0, step=0.01)

FloatSlider(value=0.17, description='Min Speed', max=1.0, step=0.01)

FloatSlider(value=0.3, description='Kp', max=10.0, step=0.01)

FloatSlider(value=0.1, description='Ki', max=10.0, step=0.01)

FloatSlider(value=0.2, description='Kd', max=10.0, step=0.01)

ToggleButton(value=True, description='STOP', icon='check', tooltip='Description')

ToggleButton(value=False, description='Snap photo', icon='check', tooltip='Description')

-0.01875
0.00625
0.015625
0.0125
0.0
-0.009375
-0.025
-0.03125
-0.01875
-0.00625
0.00625
0.009375
-0.003125
-0.015625
-0.009375
0.0375
0.159375
0.3625
0.725
0.959375
0.534375
0.371875
0.30625
0.309375
0.38125
0.509375
0.615625
0.646875
0.68125
0.659375
0.61875
0.7
0.909375
0.903125
0.653125
0.50625
0.44375
0.434375
0.465625
0.534375
0.63125
0.75
0.921875
0.890625
0.61875
0.4875
0.4375
0.45
0.515625
0.6375
0.85
0.91875
0.8375
0.75625
0.65
0.54375
0.490625
0.496875
0.525
0.6
0.696875
0.7125
0.675
0.6625
0.703125
0.778125
0.834375
0.890625
0.928125
0.94375
0.9875
0.984375
0.928125
0.95
0.99375
0.9625
0.928125
0.915625
0.90625
0.9375
0.965625
0.915625
0.928125
0.95625
0.946875
0.925
0.9125
0.9
0.875
0.859375
0.846875
0.85625
0.834375
0.83125
0.821875
0.8
0.759375
0.753125
0.7875
0.8
0.790625
0.796875
0.790625
0.759375
0.734375
0.75625
0.915625
0.9375
0.859375
0.875
0.940625
0.8875
0.909375
0.925
0.921875
0.9
0.94375
0.946875
0.88125
0.84375
0.884375
0.909375
0.890625
0.8
0.7375
0.73125
0.7

In [3]:
# Setup and export pwm0 and pwm1
pwm0 = PWM(0)
pwm1 = PWM(1)
pwm0.export()
pwm1.export()

# Set the periods to 20ms
pwm0.period = 20000000
pwm1.period = 20000000

# Set the duty_cycle of the motor to 1ms and servo to 1.5ms
setEngineSpeed(0, pwm1)
turnRight(0, pwm0)

# Enable the pwms
pwm0.enable = True
pwm1.enable = True

# Give some time to start up
time.sleep(5)

# START OF COMPUTER VISION

res = (640,480)
b, a = butter(3, 0.007)


camera = PiCamera()


# Check the link below for the combinations between mode and resolution
# https://picamera.readthedocs.io/en/release-1.13/fov.html#sensor-modes
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 60

# Initialize the buffer and start capturing
rawCapture = PiYUVArray(camera, size=res)
stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)

whiteCenterLine = 0 # Know where the center is
images = [] # Used for testing 
peaks = [] # Used for testing

# Main line following code
def func():
    global images
    turning = 'n' # Indicate which direction it is turning
    output = 0 # result of the PID
    frame_num = 6 # used for getting pictures when the button is pressed
    finLineCounter = 0 # Used to count the number of times the car passes the finish line
    oldNumsPeaks = 1
    pid = PID(Kp.value, Ki.value, Kd.value, setpoint=0) # PID used to follow the line
    pid.sample_time = 0.05
    for f in stream:
        #pid = PID(Kp.value, Ki.value, Kd.value, setpoint=0) # PID used to follow the line
        #pid.tunings(Kp.value, Ki.value, Kd.value) # Tune the PID with the slider values
        # Get the intensity component of the image (a trick to get black and white images)
        I = f.array[:, :, 0]
        # Capture images if the button is pressed
        if (button.value):
            frame_num = 0
            button.value = False
        # Capture 3 images
        if (frame_num <=2):
            images.append(I)
            frame_num = frame_num + 1
        # Reset the buffer for the next image
        rawCapture.truncate(0)

        # Select a horizontal line in the middle of the image
        ################## Lets play around with this value to see if there is a better spot
        L = I[440, :]

        # Smooth the transitions so we can detect the peaks 
        Lf = filtfilt(b, a, L)

        # Find peaks which are higher than 0.5
        p = find_peaks(Lf, height=128)
        num_peaks = len(p[0])
        # The whiteCenterLine is the value of the peak
        if num_peaks == 1:
            whiteCenterLine = p[0][0]
        # The whiteCenterLine is the average of the two widest peaks
        elif num_peaks >= 2:
            whiteCenterLine = (p[0][0] + p[0][len(p)-1]) / 2
            images.append(I) # Add the image to the list of images for review later
            peaks.append(p) # Add the peaks to the list of peaks for review later
            # finLineCounter += 1
        elif (num_peaks == 0):
            pass # REMEMBER LAST TURN
        # Print peaks
        # print(p)
        #print("num peaks ", num_peaks)
        
        # Steer the car accordingly
        # Fully turn in the same direction if we don't see the line
        if num_peaks == 0:
            if turning == 'l':
                output = 1
            elif turning == 'r':
                output = -1
        else:
            # Check that the line is not one of the edge lines. Continue on the same path if it is
            if turning == 'l' and whiteCenterLine - 320 > 150 and oldNumPeaks == 0 and False:
                pass
            elif turning == 'r' and whiteCenterLine < 150 and oldNumPeaks == 0  and False:
                pass
            else:
                # Get how far the car is off as a percent
                percentOff = (whiteCenterLine - 320) / 320.0
                print(percentOff)
                output = pid(percentOff) # Get the output of the PID as a percent
                # Update turning direction
                if whiteCenterLine < 320:
                    turning = 'l'
                else:
                    turning = 'r'
        #print("output: ", output)
        # Calculate a new speed based on the percent of turning
        newSpeed = getNewSpeed(abs(output), maxSpeed.value, minSpeed.value)
        # print("speed: ", newSpeed)
        # Make sure not to set the speed faster than maxSpeed
        if newSpeed <= maxSpeed.value:
            setEngineSpeed(newSpeed, pwm1)
        else:
            setEngineSpeed(maxSpeed.value, pwm1)
        #print("Output: ", output)
        turn(output, pwm0) # Turn the car
        oldNumPeaks = num_peaks

        # Check if the loop should end
        if not running.value:
            running.value = True
            break
       # if finLineCounter >= 1:
        #    time.sleep(0.5) # Give time to cross the finish line (May need to be changed with testing)
        #    break
    # Stop the motor
    setEngineSpeed(0, pwm1)
    # Reposition the servo to 0
    turnRight(0, pwm0)
    time.sleep(.1)

    #time_elapsed = time.time() - t
    #print("Elapsed {:0.2f} seconds, estimated FPS {:0.2f}".format(time_elapsed, N / time_elapsed))

    # Close the camera and the stream
    stream.close()
    rawCapture.close()
    camera.close()

    # Disable both pwms
    pwm0.enable = False
    pwm1.enable = False

# Start a thread for the main control. This allows for us to use the sliders/buttons
t = threading.Thread(target=func)
t.start()

0.359375
0.3625
0.359375
0.3625
0.39375
0.475
0.54375
0.6
0.621875
0.621875
0.615625
0.6
0.584375
0.584375
0.640625
-0.3875
-0.0921875
0.021875
-0.99375
-0.6
0.359375
-0.69375
-0.846875
-0.728125
-0.428125
-0.25
-0.11875
0.00625
0.08125
0.025
-0.075
-0.14375
-0.21875
-0.328125
-0.3625
-0.340625
-0.30625
-0.19375
-0.05625
0.084375
0.225
0.340625
0.39375
0.3875
0.36875
0.3875
0.471875
0.63125
0.96875
0.56875
0.340625
0.1625
0.0
-0.1375
-0.25625
-0.35
-0.434375
-0.525
-0.603125
-0.665625
-0.728125
-0.71875
-0.6625
-0.609375
-0.55
-0.48125
-0.38125
-0.271875
-0.178125
-0.115625
-0.06875
-0.0375
-0.034375
-0.046875
-0.04375
-0.0125
0.025
0.071875
0.121875
0.16875
0.20625
0.2375
0.275
0.325
0.375
0.4375
0.465625
0.453125
0.440625
0.409375
0.346875
0.25
0.13125
0.028125
-0.05625
-0.134375
-0.184375
-0.2125
-0.196875
-0.1375
-0.0375
0.084375
0.259375
0.459375
0.65625
0.896875
0.6875
0.4
0.19375
0.040625
-0.0375
-0.01875
0.096875
0.284375
0.5
0.815625
0.59375
0.38125
0.28125
0.2625
0.31875
0.38

In [ ]:
stream.close()
rawCapture.close()
camera.close()

In [ ]:
for i in range(0,len(images)): 
    plt.imshow(images[i])

plt.figure(figsize=(18, 24))

for k, img in enumerate(images):
    ax = plt.subplot(4, 3, k + 1)
    ax.imshow(img)
    
plt.tight_layout()

In [ ]:
len(images)

In [ ]:
plt.imshow(images[0])

In [ ]:
L = images[1][440, :]

# Smooth the transitions so we can detect the peaks 
Lf = filtfilt(b, a, L)

# Find peaks which are higher than 0.5
p = find_peaks(Lf, height=128)
print(p)